In [357]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.layers import Input, Dense, concatenate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from keras.models import Sequential

In [358]:
# loading data
data = pd.read_csv("C:\\Users\\xinga\\OneDrive\\文档\\GitHub\\AI-Project\\dataset\\titanic.csv")

data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [359]:
null_rows = data[data.isnull().any(axis=1)]
print(null_rows)

     PassengerId  Survived  Pclass                                      Name  \
0              1         0       3                   Braund, Mr. Owen Harris   
2              3         1       3                    Heikkinen, Miss. Laina   
4              5         0       3                  Allen, Mr. William Henry   
5              6         0       3                          Moran, Mr. James   
7              8         0       3            Palsson, Master. Gosta Leonard   
..           ...       ...     ...                                       ...   
884          885         0       3                    Sutehall, Mr. Henry Jr   
885          886         0       3      Rice, Mrs. William (Margaret Norton)   
886          887         0       2                     Montvila, Rev. Juozas   
888          889         0       3  Johnston, Miss. Catherine Helen "Carrie"   
890          891         0       3                       Dooley, Mr. Patrick   

        Sex   Age  SibSp  Parch        

In [360]:
data = data.drop(["PassengerId","Name","Ticket","Cabin"], axis=1) # drop the column not useful
data["Age"] = data["Age"].fillna(data["Age"].median()) # fill null value using median value
data["Fare"].fillna(data["Fare"].median())
# data = data.dropna('Embarked', axis=1)
data = data.dropna(subset=['Embarked'])


In [361]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Sex       889 non-null    object 
 3   Age       889 non-null    float64
 4   SibSp     889 non-null    int64  
 5   Parch     889 non-null    int64  
 6   Fare      889 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 62.5+ KB


In [362]:
data = pd.get_dummies(data,columns=["Sex","Embarked"]) # convert text data to numerical format. to_categorical() can only handle
data

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,0,1,0,0,1
887,1,1,19.0,0,0,30.0000,1,0,0,0,1
888,0,3,28.0,1,2,23.4500,1,0,0,0,1
889,1,1,26.0,0,0,30.0000,0,1,1,0,0


In [363]:
x = data.drop(['Survived'],axis = 1).values
y = data["Survived"].values

In [364]:
x.shape

(889, 10)

In [365]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)

In [366]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(711, 10)
(178, 10)
(711,)
(178,)


In [367]:
# define the K-fold cross-validation parameters
num_folds = 3
kfold = KFold(n_splits=num_folds, shuffle=True) # initialize
# shuffle=True option randomized the order of thre data

In [368]:
def create_model():
    model = Sequential()
    model.add(Dense(8, input_dim = x.shape[1],activation = 'relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))# output of this layer will be probability betweent 0 and 1
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [369]:
# Train and evaluate the model using K-fold cross_validation
acc = []
loss = []
for train_index, test_index in kfold.split(x_train):
    model = create_model()
    model.fit(x_train[train_index],y_train[train_index],epochs=20,batch_size=32, verbose=0)
    score = model.evaluate(x[test_index],y[test_index],verbose=0)
    print(score)
    acc.append(score[1])
    loss.append(score[0])

[0.8334136009216309, 0.6455696225166321]
[0.6846793293952942, 0.6371307969093323]
[0.605830729007721, 0.6751055121421814]


In [370]:
print("mean accuracy: %.2f%%" %(np.mean(acc)*100))
print("mean loss: %.2f%%" %(np.mean(loss)*100))


mean accuracy: 65.26%
mean loss: 70.80%
